In [1]:
import pandas as pd
import numpy as np

In [2]:

members = [
    '../data/members/AMIC_ Digital.xlsx',   
    '../data/members/AMIC_ Papel.xlsx',
    '../data/members/ROW AMIC Papel + digital.xlsx'
]
sources = [
    '../data/sources/cac/DadesMapa-CAC.xlsx',
    '../data/sources/iberfier/iberifier_ procesado.xlsx',
    '../data/sources/iberfier/ROW iberifier.csv',
    '../data/sources/ojd/OJD DOWNLOAD.xlsx'
]

In [3]:
region_map = {
    'IB': ['balears', 'illes balears', 'balearic islands', 'balears; illes'],
    'CAT': ['catalunya', 'cataluña', 'catalonia'],
    'PV': ['país valencià', 'comunitat valenciana', 'valencian community', 'valencia'],
    'AD': ['andorra'],
    'ARA': ['aragó', 'aragon', 'aragón'],
    'SARDENYA': ['sardenya', 'sardinia'],
    'CAT NORD': ['catalunya nord', 'cat nord', 'catnord', 'cataluña nord'],
    'AN': ['andalusia', 'andalucía'],
    'AST': ['astúries', 'asturias'],
    'CANT': ['cantàbria', 'cantabria'],
    'CL': ['castella i lleó', 'castilla y león', 'castile and leon'],
    'CLM': ['castella-la manxa', 'castilla-la mancha', 'castile-la mancha'],
    'CEU': ['ceuta'],
    'CYM': ['canàries', 'canarias', 'canary islands'],
    'EXT': ['extremadura'],
    'GAL': ['galícia', 'galicia'],
    'MAD': ['madrid'],
    'MEL': ['melilla'],
    'MUR': ['múrcia', 'murcia'],
    'NAV': ['navarra', 'navarre'],
    'LR': ['la rioja'],
    'EUS': ['euskal herria', 'euskadi', 'país basc', 'basque country', 'país vasco']
}
def get_ccaa(region):
    for ccaa, regions in region_map.items():
        for r in regions:
            if r in region.lower():
                return ccaa
    return None


#### 1. Digital Sheet

In [4]:
def fetch_digital():
    digital = pd.read_excel(members[0], 'AMIC - Digital', skiprows=1, usecols=['Región', 'Mitjà', 'zona d\'influència', 'Región Origen', 'Area', 'OJD si/ no'])
    return clean_digital(digital)

def clean_digital(df: pd.DataFrame):
    df = df.rename(columns={
        'Región': 'region',
        'Mitjà': 'media',
        'zona d\'influència': 'area_of_influence',
        'Región Origen': 'province',
        'Area': 'area',
        'OJD si/ no': 'ojd'
    })
    df['region'].fillna('', inplace=True)
    df.dropna(subset=['media'], inplace=True)
    df['url'] = df['media'].apply(lambda x: x.split('//')[-1].split('/')[0])
    df['media'] = df['url'].str.split('.').str[0]
    df['CCAA'] = df['region'].apply(get_ccaa)
    df['PROV'] = df['province'].fillna('').apply(lambda x: x.replace(' ', '').replace('/', '').strip())
    df['AREA'] = df['area'].fillna('').apply(lambda x: x.replace(' ', '').replace('/', '').strip())

    df.drop(columns=['region', 'province', 'area'], inplace=True)
    df['platform'] = 'digital'
    df.dropna(subset=['media'], inplace=True)
    return df

#### 2. Papel Sheet

In [5]:
def fetch_papel():
    papel = pd.read_excel(members[1], 'AMIC - Paper', usecols=['CCAA', 'PROV', 'ÀREA', 'PGD', 'mitjà', 'Area influencia', 'Distribució', 'CCAA2', 'PROV3', 'ÀREA4'])
    return clean_papel(papel)

def clean_papel(df: pd.DataFrame):
    df = df.rename(columns={
        'CCAA': 'region',
        'PROV': 'province',
        'ÀREA': 'area',
        'mitjà': 'media',
        'Area influencia': 'area_of_influence',
        'Distribució': 'distribution',
        'CCAA2': 'region2',
        'PROV3': 'province2',
        'ÀREA4': 'area2'
    })
    df.dropna(subset=['media'], inplace=True)
    df['media'] = df['media'].apply(lambda x: x.lower().strip())
    df['region'].fillna('', inplace=True)
    df['CCAA'] = df['region'].apply(get_ccaa)
    df['CCAA'].fillna(df['region2'], inplace=True)
    df['PROV'] = df['province2'].fillna('').apply(lambda x: x.replace(' ', '').replace('/', '').strip())
    df['AREA'] = df['area2'].fillna('').apply(lambda x: x.replace(' ', '').replace('/', '').strip())
    df.drop(columns=['region2', 'province2', 'area2', 'province', 'area', 'region'], inplace=True)

    df['platform'] = 'papel'
    return df


In [19]:
def fetch_iber():
    iber = pd.read_excel(sources[1], 'iberfier (2)', usecols=['TITLE', 'ACTIVE', 'URL_web', 'platforms', 'province', 'region', 'LOCATION_company_address', 'LOCATION_company_zipcode', 'lat', 'lng'],
                         dtype={'LOCATION_company_zipcode': str})
    return clean_iber(iber)


def clean_iber(df: pd.DataFrame):
    df = df.rename(columns={
        'TITLE': 'media',
        'ACTIVE': 'active_iber',
        'URL_web': 'url',
        'platforms': 'platform_iber',
        'LOCATION_company_address': 'address_iber',
        'LOCATION_company_zipcode': 'zipcode_iber',
        'lat': 'lat_iber',
        'lng': 'lng_iber',
        'province': 'province_iber',
    })
    df.dropna(subset=['media'], inplace=True)
    df['media'] = df['media'].apply(lambda x: x.lower().strip())

    df['region'].fillna('', inplace=True)
    df['CCAA'] = df['region'].apply(get_ccaa)
    df['url'] = df['url'].apply(lambda x: x.split('//')[-1].split('/')[0].split('www.')[-1])

    df.drop(columns=['region'], inplace=True)
    return df

In [21]:
fetch_iber()['zipcode_iber']

0        NaN
1       8036
2       8302
3        NaN
4       8021
       ...  
699    25230
700    12071
701     8202
702     8006
703    46800
Name: zipcode_iber, Length: 704, dtype: object

## 4. CAC Data

In [7]:

def fetch_cac():
    cac = pd.read_excel(sources[0], 'Mapa CAC', usecols=['Nom comercial', 'Mitjà', 'Domicili', 'Latitud', 'Longitud', 'Web'])
    return clean_cac(cac)

def clean_cac(df: pd.DataFrame):
    df = df.rename(columns={
        'Nom comercial': 'media',
        'Mitjà': 'type_media_cac',
        'Domicili': 'address_cac',
        'Latitud': 'lat_cac',
        'Longitud': 'lng_cac',
        'Web': 'url'
    })
    df.dropna(subset=['media'], inplace=True)
    df = df[df['media'] != '-']
    df['url'] = df['url'].apply(lambda x: x.split('//')[-1].split('/')[0] if pd.notnull(x) else None)
    # extract zip code from address -> <5-6 digits>
    df['zipcode_cac'] = df['address_cac'].str.extract(r'(\d{5,6})')
    df['media'] = df['media'].apply(lambda x: x.lower().strip())

    return df



## 5. OJD Data

In [8]:

'''['Comunidad', 'Clasificación', 'TITULO', 'CLASIFICACION',
       'SUB-CLASIFICACION', 'N.UNICOS(avg)', 'N.UNICOS', 'VAR.%', 'VISITAS',
       'D.MEDIA', 'PAGINAS'],'''
def fetch_ojd():
    ojd = pd.read_excel(sources[3], 'OJDinteractiva-Ultimos-Medios-A', skiprows=4, usecols=['TITULO', 'CLASIFICACION', 'SUB-CLASIFICACION', 'N.UNICOS(avg)'])
    return clean_ojd(ojd)

def clean_ojd(df: pd.DataFrame):
    df = df.rename(columns={
        'TITULO': 'media',
        'CLASIFICACION': 'classification_ojd',
        'SUB-CLASIFICACION': 'sub_classification_ojd',
        'N.UNICOS(avg)': 'unique_visitors_ojd'
    })
    df.dropna(subset=['media'], inplace=True)
    df['url'] = df['media'].apply(lambda x: x.lower().strip().split('//')[-1].split('/')[0])
    df['media'] = df['url'].str.split('.').str[0]

    return df

    

###  main( )

In [9]:
digital = fetch_digital()
papel = fetch_papel()
stacked = pd.concat([digital, papel], ignore_index=True)
stacked['Origin'] = 'AMIC'
iber = fetch_iber()
cac = fetch_cac()
ojd = fetch_ojd()

#### amic + iber

In [10]:
merged_url = pd.merge(stacked, iber, on='url', how='inner', suffixes=('', '_iber'))     # no media nulls
merged_url['Origin'] = merged_url['Origin'] + ', Iberfier'

stacked_unmatched = stacked[~stacked['media'].isin(merged_url['media'])]

merged_media = pd.merge(stacked_unmatched, iber, on='media', how='inner', suffixes=('', '_iber'))        # no media nulls
merged_media['Origin'] = merged_media['Origin'] + ', Iberfier'

stacked_unmatched_final = stacked_unmatched[~stacked_unmatched['media'].isin(merged_media['media'])]

iber_unmatched = iber[~iber['media'].isin(pd.concat([merged_url['media_iber'], merged_media['media']]))]
iber_unmatched_final = iber_unmatched[~iber_unmatched['url'].isin(pd.concat([merged_url['url'], merged_media['url_iber']]))]

iber_unmatched_final = iber_unmatched_final.rename(columns={'CCAA': 'CCAA_iber'})
iber_unmatched_final['Origin'] = 'Iberfier'

ibam = pd.concat([merged_url, merged_media, stacked_unmatched_final, iber_unmatched_final], ignore_index=True).reset_index(drop=True)
ibam.drop(columns=['url_iber', 'media_iber'], inplace=True)

### amic + iber + cac

In [11]:
ibam_url = pd.merge(ibam, cac, on='url', how='inner', suffixes=('', '_cac'))     # no media nulls
ibam_url['Origin'] = ibam_url['Origin'] + ', CAC'

ibam_unmatched = ibam[~ibam['media'].isin(ibam_url['media'])]

ibam_media = pd.merge(ibam_unmatched, cac, on='media', how='inner', suffixes=('', '_cac'))        # no media nulls
ibam_media['Origin'] = ibam_media['Origin'] + ', CAC'

ibam_unmatched_final = ibam_unmatched[~ibam_unmatched['media'].isin(ibam_media['media'])]

cac_unmatched = cac[~cac['media'].isin(pd.concat([ibam_url['media_cac'], ibam_media['media']]))]
cac_unmatched_final = cac_unmatched[~cac_unmatched['url'].isin(pd.concat([ibam_url['url'], ibam_media['url_cac']]))]

cac_unmatched_final = cac_unmatched_final.rename(columns={'CCAA': 'CCAA_cac'})
cac_unmatched_final['Origin'] = 'CAC'

ibamc = pd.concat([ibam_url, ibam_media, ibam_unmatched_final, cac_unmatched_final], ignore_index=True).reset_index(drop=True)
ibamc.drop(columns=['url_cac', 'media_cac'], inplace=True)

### amic + iber + cac + ojd

In [12]:
ibamc_url = pd.merge(ibamc, ojd, on='url', how='inner', suffixes=('', '_ojd'))     # no media nulls
ibamc_url['Origin'] = ibamc_url['Origin'] + ', OJD'

ibamc_unmatched = ibamc[~ibamc['media'].isin(ibamc_url['media'])]

ibamc_media = pd.merge(ibamc_unmatched, ojd, on='media', how='inner', suffixes=('', '_ojd'))        # no media nulls
ibamc_media['Origin'] = ibamc_media['Origin'] + ', OJD'

ibamc_unmatched_final = ibamc_unmatched[~ibamc_unmatched['media'].isin(ibamc_media['media'])]

ojd_unmatched = ojd[~ojd['media'].isin(pd.concat([ibamc_url['media'], ibamc_media['media']]))]
ojd_unmatched_final = ojd_unmatched[~ojd_unmatched['media'].isin(pd.concat([ibamc_url['media'], ibamc_media['media']]))]

ojd_unmatched_final['Origin'] = 'OJD'

ibamco = pd.concat([ibamc_url, ibamc_media, ibamc_unmatched_final, ojd_unmatched_final], ignore_index=True).reset_index(drop=True)
ibamco.drop(columns=['media_ojd', 'url_ojd'], inplace=True)

In [13]:
ibamco.Origin.value_counts()

Origin
AMIC, CAC              1656
AMIC, Iberfier, CAC     564
Iberfier                380
CAC                     211
Iberfier, CAC           189
AMIC                    151
OJD                     149
AMIC, Iberfier          145
AMIC, Iberfier, OJD      97
AMIC, OJD                77
Iberfier, OJD            48
AMIC, CAC, OJD           24
Iberfier, CAC, OJD        5
Name: count, dtype: int64

In [14]:
ibamco[['media', 'Origin']]

,media,Origin
0,el 9 nou,"Iberfier, CAC, OJD"
1,el 9 nou,"Iberfier, CAC, OJD"
2,betevé,"Iberfier, CAC, OJD"
3,pirineus tv,"Iberfier, CAC, OJD"
4,ràdio sabadell,"Iberfier, CAC, OJD"
...,...,...
3691,urgelltv,OJD
3692,21radio,OJD
3693,aue,OJD
3694,fembase,OJD


In [15]:
ibamco

,media,area_of_influence,ojd,url,CCAA,PROV,AREA,platform,PGD,distribution,...,province_iber,CCAA_iber,type_media_cac,address_cac,lat_cac,lng_cac,zipcode_cac,classification_ojd,sub_classification_ojd,unique_visitors_ojd
0,el 9 nou,NaN,NaN,el9nou.cat,NaN,NaN,NaN,NaN,NaN,NaN,...,Barcelona,CAT,Emissora de ràdio,"Plaça de la Catedral, 2 , 08500, Vic",41.928026,2.254678,08500,Notícies i Informació,Notícies globals i actualitat,8693.0
1,el 9 nou,NaN,NaN,el9nou.cat,NaN,NaN,NaN,NaN,NaN,NaN,...,Barcelona,CAT,Canal de televisió,"Carrer Santa Maria, 4 , 08500, Vic",41.927802,2.254682,08500,Notícies i Informació,Notícies globals i actualitat,8693.0
2,betevé,NaN,NaN,beteve.cat,NaN,NaN,NaN,NaN,NaN,NaN,...,Barcelona,CAT,Canal de televisió,"Plaça Tísner, 1 , 08099, Barcelona",41.400574,2.193725,08099,Entreteniment,Broadcast,59925.0
3,pirineus tv,NaN,NaN,pirineustv.cat,NaN,NaN,NaN,NaN,NaN,NaN,...,Lleida,CAT,Canal de televisió,"Avinguda Pau Claris, 8 , 25700, La Seu d'Urgell",42.358465,1.459471,25700,Notícies i Informació,Notícies globals i actualitat,226.0
4,ràdio sabadell,NaN,NaN,radiosabadell.fm,NaN,NaN,NaN,NaN,NaN,NaN,...,Barcelona,CAT,Emissora de ràdio,"Carrer Dr. Puig, 18, , 08202, Sabadell",41.549914,2.109091,08202,Entreteniment,Broadcast,3216.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3691,urgelltv,NaN,NaN,urgelltv.cat,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Notícies i Informació,Notícies globals i actualitat,15.0
3692,21radio,NaN,NaN,21radio.cat,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entreteniment,Broadcast,13.0
3693,aue,NaN,NaN,aue.cat,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Notícies i Informació,Notícies globals i actualitat,8.0
3694,fembase,NaN,NaN,fembase.cat,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Notícies i Informació,Noticies deportives y actualitat,6.0


In [16]:
ibamco.to_csv('../data/ibamco.csv', index=False)

In [336]:
48+74+68+380+45+142

757

In [337]:
ibamco[['Origin', 'media']].drop_duplicates().groupby('Origin').count().sum()

media    1371
dtype: int64

In [18]:
ibamco[ibamco.zipcode_cac != ibamco.zipcode_iber][['zipcode_cac', 'zipcode_iber']]

,zipcode_cac,zipcode_iber
0,08500,8500
1,08500,8500
2,08099,8018
4,08202,8202
5,NaN,8500
...,...,...
3691,NaN,NaN
3692,NaN,NaN
3693,NaN,NaN
3694,NaN,NaN
